In [50]:
!pip install -q crewai langchain langchain_google-genai langchain_community langchain_core langchain_mistralai duckduckgo-search wikipedia langchain_experimental arxiv faiss-cpu langchainhub

In [75]:
from crewai import Agent , Task , Crew , Process
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools import PythonREPLTool
from langchain.tools import DuckDuckGoSearchRun ,WikipediaQueryRun
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.utilities import WikipediaAPIWrapper
import subprocess
import duckduckgo_search
import pprint
from langchain.retrievers import ArxivRetriever
from langchain_community.document_loaders import TextLoader

In [76]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [77]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
@tool
def random(a):
    """Look up things online."""
    return "LangChain"
random_tool= Tool(
    name="random",
    description="Use must use this function whenever you are asked to print something random",
    func=random,
)

In [78]:
# wiki_tool=WikipediaQueryRun()
py_tool=PythonREPLTool()
search_tool=DuckDuckGoSearchRun()
# retriever = ArxivRetriever(load_max_docs=2)
# docs = retriever.get_relevant_documents(query="1605.08386")
loader = TextLoader("/content/untitled.txt")
documents = loader.load()
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_agents",
    "Searches and returns data about langchain agents.",
)
tools = [retriever_tool,py_tool,search_tool,random_tool]

In [79]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

In [80]:
model=ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [81]:
from langchain.agents import AgentExecutor, create_openai_tools_agent , initialize_agent ,create_react_agent
agent = create_react_agent(model, tools, prompt)

In [82]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,max_iterations=3,early_stopping_method='generate',)

In [86]:
print(agent_executor.invoke(
    {
        "input": "print something random"
    }
))



> Entering new AgentExecutor chain...
I should just call the random module to print a random value
Action: random
Action Input: random.randint(1, 100)LangChainthe final answer should be LangChain
Final Answer: LangChain

> Finished chain.
{'input': 'print something random', 'output': 'LangChain'}
